# anotacao dos solar panels um a um

**tens de mudar estes caminhos**

1. caminho para a pasta que tem os poligonos todos que marcadas

2. caminho para a pasta que vai ter os ficheiros yolo (é só criar a pasta e meter o .yaml lá dentro, nada mais)

In [ ]:
path_to_new_polygons_jsonfolder = "ModelBuild/model13_polygons" # pasta q tem os arquivos json
path_to_yolo_files = "ModelBuild/model13" # pasta q vai ter yaml, val folder, train folder

preparar os dados

In [ ]:
import pandas as pd

df_train = pd.read_pickle('Model_Train.pkl')
df_val = pd.read_pickle('Model_Val.pkl')

# merge train and val
df_train = pd.concat([df_train, df_val], ignore_index=True).reset_index(drop=True)
df_train.shape

In [ ]:
df_train.sample(3)

verificar multiplos paneis em poligonos

In [ ]:
def has_multi_panel(nr_boil, nr_pan):
    return any(x > 1 for x in nr_boil) or any(x > 1 for x in nr_pan)

# remove multi panel images
df_train["multi_panel"] = df_train.apply(lambda row: has_multi_panel(row["nr_boil"], row["nr_pan"]), axis=1)

print("Current train set status: ")
df_train["multi_panel"].value_counts()

atualizar novos poligonos no dataset

In [ ]:
import json
import os


for filename in os.listdir(path_to_new_polygons_jsonfolder):
    if filename.endswith(".json"):
        with open(f'{path_to_new_polygons_jsonfolder}/{filename}') as f:
            data = json.load(f)

        img_id = filename.split(".")[0]
        nr_boil = []
        polygons_boil = []
        nr_pan = []
        polygons_pan = []
        multi_panel = False
        
        polygons = data["shapes"]
        image_size = data["imageWidth"], data["imageHeight"]

        for polygon in polygons:
            label = polygon["label"]
            points = polygon["points"]
            new_polygon = []
            for x,y in points:
                x = (x*512)/image_size[0]
                y = (y*512)/image_size[1]
                new_polygon.append((x,y))

            if "boil" in polygon["label"]:
                nr_boil.append(1)
                polygons_boil.append(new_polygon)
        
            elif "pan" in polygon["label"]:
                nr_pan.append(1)
                polygons_pan.append(new_polygon)

            else:
                print("Unknown label")

        # img_id index
        index = df_train.loc[df_train["img_id"] == img_id].index[0]

        df_train.at[index, "nr_boil"] = nr_boil
        df_train.at[index, "polygons_boil"] = polygons_boil
        df_train.at[index, "nr_pan"] = nr_pan
        df_train.at[index, "polygons_pan"] = polygons_pan
        df_train.at[index, "multi_panel"] = multi_panel

df_train.sample(3)

remover instances com multi panels

In [ ]:
df_train = df_train[df_train["multi_panel"] == False].reset_index(drop=True)
df_train.shape

voltar a rebalancear o dataset em treino e val

In [ ]:
df = df_train.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into 80% train and 20% test
train_size = int(0.8 * len(df))
train_df = df[:train_size]
val_df = df[train_size:]

print("Train size:", len(train_df))
print("Test size:", len(val_df))

percorrer a df: resize images, create masks, ...

In [ ]:
import numpy as np
import cv2

def polygons_to_segmentation_mask(polygon, image, label):
    if label == "pan":
        val = 2
    elif label == "boil":
        val = 1
    else:
        print("Unknown label")

    poly = np.array([polygon], dtype=np.int32)
    cv2.fillPoly(image, [poly], val)     
    return image

def resize_image(name, size=(512, 512)):
    image_file = f"images/{name}.jpg"
    image = cv2.imread(image_file)
    resized_image = cv2.resize(image, size)
    return resized_image

In [ ]:
from ultralytics.data.converter import convert_segment_masks_to_yolo_seg

def process_dataframe(df, path_to_):
    
    # criar pasta para imagens e mascaras
    os.makedirs(f"{path_to_}/images", exist_ok=True)
    os.makedirs(f"{path_to_}/masks", exist_ok=True)
    os.makedirs(f"{path_to_}/labels_sep", exist_ok=True)
    os.makedirs(f"{path_to_}/labels", exist_ok=True)

    # criar mascaras para cada pan e para cada boil
    for index, row in df.iterrows():
        img_name = row["img_id"]

        resized_image = resize_image(img_name)
        cv2.imwrite(f"{path_to_}/images/{img_name}.png", resized_image)

        for i, (nr, polygon) in enumerate(zip(row["nr_boil"], row["polygons_boil"])):
            mask = np.zeros((512, 512), np.uint8)
            mask = polygons_to_segmentation_mask(polygon, mask, "boil")
            cv2.imwrite(f"{path_to_}/masks/{img_name}_boil_{i}.png", mask)
        
        for i, (nr, polygon) in enumerate(zip(row["nr_pan"], row["polygons_pan"])):
            mask = np.zeros((512, 512), np.uint8)
            mask = polygons_to_segmentation_mask(polygon, mask, "pan")
            cv2.imwrite(f"{path_to_}/masks/{img_name}_pan_{i}.png", mask)

    # converter cada mascara para yolo format   
    convert_segment_masks_to_yolo_seg(masks_dir=f"{path_to_}/masks",
                                      output_dir=f"{path_to_}/labels_sep", classes=2)

    # agrupar yolo formats que sejam da mesma imagem
    for filename in os.listdir(f"{path_to_}/labels_sep"):
        if filename.endswith(".txt"):
            img_id = filename.split("_")[0]
            with open(f"{path_to_}/labels_sep/{filename}", "r") as f:
                lines = f.readlines()

            with open(f"{path_to_}/labels/{img_id}.txt", "a") as f:
                for line in lines:
                    f.write(line)

process_dataframe(train_df, path_to_yolo_files + "/train")
process_dataframe(val_df, path_to_yolo_files + "/val")

start yolo model

In [ ]:
from ultralytics import YOLO


# Load YOLOv8 Model
model = YOLO("yolo11m-seg")

# Train the model
model.train(
    data=f'{path_to_yolo_files}/data.yaml',  # Path to your data.yaml
    epochs=75,                           # Number of epochs
    imgsz=512,                           # Image size
    batch=32,                            # Batch size
    augment=True,                        # Enable data augmentation
    project=f'{path_to_yolo_files}',                # Path where training results will be saved (logs, checkpoints)
    name='m13_yolo',                 # Name of the specific experiment folder (creates a new folder)
    exist_ok=True,                         # Overwrite the folder if it already exists
    patience=5,                          # Stop training if loss does not improve for this number of consecutive epochs
)

---

In [1]:
from ultralytics import YOLO

model = YOLO("ModelBuild/model13_yolo/weights/best.pt")

#metrics = model.val()

---

In [4]:
import pandas as pd
import cv2

test = pd.read_csv("Test.csv")
submission = pd.read_csv("zindi_files/SampleSubmission.csv")

def resize_image(name, size=(512, 512)):
    image_file = f"images/{name}.jpg"
    image = cv2.imread(image_file)
    resized_image = cv2.resize(image, size)
    cv2.imwrite(f"ModelBuild/model13_subm/{name}.png", resized_image)

for i, row in test.iterrows():
    if i % 100 == 0:
        print(i, len(test))

    resize_image(row["ID"])
    results = model(f"ModelBuild/model13_subm/{row['ID']}.png")
    result = results[0]

    # Count occurrences of each detected class
    class_counts = {}

    for cls in result.boxes.cls.tolist():  # Get list of detected class indices
        class_counts[int(cls)] = class_counts.get(int(cls), 0) + 1

    # Map class indices to class names
    class_names = model.names  # Dictionary mapping class index to name
    named_counts = {class_names[k]: v for k, v in class_counts.items()}

    if "boil" in named_counts:
        submission.loc[submission["ID"] == f'{row["ID"]}_boil', "Target"] = named_counts["boil"]
    
    if "pan" in named_counts:
        submission.loc[submission["ID"] == f'{row["ID"]}_pan', "Target"] = named_counts["pan"]

submission.head(10)

0 1107

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model13_subm/ID00qprY.png: 512x512 1 pan, 386.0ms
Speed: 1.0ms preprocess, 386.0ms inference, 1.7ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model13_subm/ID01AciUc.png: 512x512 1 pan, 326.8ms
Speed: 0.8ms preprocess, 326.8ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model13_subm/ID0328D.png: 512x512 2 pans, 312.5ms
Speed: 0.6ms preprocess, 312.5ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model13_subm/ID05WxObCFTs9.png: 512x512 3 pans, 333.3ms
Speed: 0.8ms preprocess, 333.3ms inference, 1.9ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model13_subm/ID06AdCmLMlkO.png: 512x512 2 boils, 324.1ms
Speed: 0.7ms preprocess, 324.1ms inference, 1.7ms postprocess per image at shape (

,ID,Target
0,ID00qprY_boil,0
1,ID00qprY_pan,1
2,ID01AciUc_boil,0
3,ID01AciUc_pan,1
4,ID0328D_boil,0
5,ID0328D_pan,2
6,ID05WxObCFTs9_boil,0
7,ID05WxObCFTs9_pan,3
8,ID06AdCmLMlkO_boil,2
9,ID06AdCmLMlkO_pan,0


In [5]:
submission.to_csv("zindi_files/model13_subm.csv", index=False)